In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

## User input

In [ ]:
# path where segmentation measurements are stored
fn_measurements = Path(
    r'Z:\zmbstaff\9309\Raw_Data\240531_Inhibitor_screening_1\converted_data\measurements.csv'
)
# path to where the FLIManalysis fit results are stored (Results_byROI.xlsx)
fn_results_xlsx = Path(
    r'Z:\zmbstaff\9309\Raw_Data\240531_Inhibitor_screening_1\converted_data\roiExtract\Results\mono_exp\Results_byROI.xlsx'
)

# path & filename for the combined output table
fn_output = Path(
    r'Z:\zmbstaff\9309\Raw_Data\240531_Inhibitor_screening_1\converted_data\output_table.csv'
)

## Load & combine data

In [ ]:
# load segmentation measurements
df_measurements = pd.read_csv(fn_measurements)

In [ ]:
# load fit-results
df_results = pd.read_excel(fn_results_xlsx, sheet_name=None)

# add ROI_names to df
for ROI_name, df in df_results.items():
    df["label"] = int(ROI_name.split("_")[1])
    df.insert(1, "label", df.pop("label"))

# merge sheets
df_results = pd.concat([df_results[ROI_name] for ROI_name in df_results.keys()], ignore_index=True)
# sort by image and label
df_results = df_results.sort_values(['Image','label'])
# remove rows with NaN in iem column (these should be the ones without a label)
df_results = df_results[~df_results.iem.isna()]
# reset index
df_results = df_results.reset_index(drop=True)
# add prefix
df_results = df_results.add_prefix('fit_')

In [ ]:
# COMBINE SEGMENTATION-MEASUREMENTS AND FIT-RESULTS

df = pd.concat([df_measurements, df_results], axis=1)

# check that data is consistent between the two files
if not all(df.label == df.fit_label):
    raise ValueError("ERROR: labels do not match")
for scene in df.scene.unique():
    df_scene = df[df.scene == scene]
    if len(df_scene.fit_Image.unique()) != 1:
        raise ValueError(f"ERROR: images do not match")
    
# add well information
df["well"] = df.scene.str.extract(r"([A-Z][0-9]{2})")
df.insert(0, "well", df.pop("well"))

# clean up
df.pop("fit_Image")
df.pop("fit_label")

df

## Export combined data

In [ ]:
os.makedirs(fn_output.parent, exist_ok=True)
df.to_csv(fn_output, index=False)

## Plot some things

In [ ]:
x_col = "fit_tau_1"
y_col = "fit_chi2"
plt.scatter(df[x_col], df[y_col], s=16, alpha=0.25, edgecolors='none')
plt.xlabel(x_col)
plt.ylabel(y_col)

In [ ]:
x_col = "fit_I"
y_col = "fit_chi2"
plt.scatter(df[x_col], df[y_col], s=16, alpha=0.25, edgecolors='none')
plt.xlabel(x_col)
plt.ylabel(y_col)

In [ ]:
x_col = "AD_ratio"
y_col = "fit_tau_1"
plt.scatter(df[x_col], df[y_col], s=16, alpha=0.25, edgecolors='none')
plt.xlabel(x_col)
plt.ylabel(y_col)

In [ ]:
x_col = "donor_intensity_mean"
y_col = "fit_tau_1"
plt.scatter(df[x_col], df[y_col], s=16, alpha=0.25, edgecolors='none')
plt.xlabel(x_col)
plt.ylabel(y_col)

In [ ]:
x_col = "acceptor_intensity_mean"
y_col = "fit_tau_1"
plt.scatter(df[x_col], df[y_col], s=16, alpha=0.25, edgecolors='none')
plt.xlabel(x_col)
plt.ylabel(y_col)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

df.boxplot(
    column='fit_tau_1',
    by='well',
    ax=ax,
    notch=True,
    grid=False,
)
ax.set_ylabel('fit_tau_1')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

plt.violinplot(dataset=[df[df['well'] == well]['fit_tau_1'] for well in df['well'].unique()], positions=range(len(df['well'].unique())), showmedians=True)
ax.set_xticks(range(len(df['well'].unique())))
ax.set_xticklabels(df['well'].unique())
ax.set_ylabel('fit_tau_1')
